# Spike proteinを開裂するproteasesの基質たちを学習して，新たなSpike protein を開裂するproteases を探す機械学習モデルを作成する．

# merops からcopy&pasteで作ったproteaseの基質たちの情報が入ったファイルの名前を配列に格納します．
ここもいずれは自動化したい．
おそらく，merops databaseを構築して，そこから情報を検索するようにすればいいのかと．

2022/07/10現在
<br>
MEROPSのDBは構築済みなので，Uniprot IDのリストをMEROPS DBから作成すればよくなりました．
<br>
Web上のMEROPSでは存在する基質データが構築したDBにはまだ反映されていなくて存在しない例もあるようなので注意です．

In [1]:
#1
import re
import numpy as np
import glob
import random
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from urllib.request import urlopen
from lxml import etree

MEROPSはmysqlでもう動くので本当はここは使いたくない．
2022/07/26現在訂正します．MySQLによるMROPSデータベースでは，更新が間に合っていないデータがあるので，ウェブからコピペで取得したものを使うべきです．たった４つしかポジティブデータがないので，手動でできる．この時，ファイルの内容をDataframeに変換して扱います．

In [2]:
filenames_posi = []
filenames_posi = glob.glob("trainingdata_positive/*")

In [3]:
print(filenames_posi)

[]


# mysqlに接続します．

In [4]:
#ここからはじまる．
import mysql.connector as mydb

# コネクションの作成
conn = mydb.connect(
    host='localhost',
    port='3306', #文字列型ですが，int型にすべきときもあります．
    user='XXXXXXX', #MySQL内でのユーザー名
    password='XXXXXXXX',　#ユーザーパスワード
    database='meropsrefs01'
)

In [5]:
# DB操作用にカーソルを作成
cur = conn.cursor()

# MySQLで使うMEROPS code辞書作成

HATL1は開裂する基質情報が登録されていなかった．（2022年7月24日現在）<br>
TMPRSS13はMSPLとも呼ぶ． MSPLは4つの基質情報があったが，Uniprot IDが記載されていなかった．(2022年7月24日現在)

In [6]:
positive_proteases = {"HAT":"S01.047", "DESC1":"S01.021", "HATL1":"S01.292", 
                      "TMPRSS2":"S01.247", "TMPRSS4":"S01.034", "TMPRSS13":"S01.087"}

In [7]:
print(positive_proteases["HAT"])

S01.047


In [8]:
merops_code = [positive_proteases["HAT"]]
cur.execute("SELECT uniprot_acc, p1 FROM cleavage WHERE code=(%s);", merops_code)

In [9]:
a = cur.fetchall()
print(a)

[('A3R530', 320), ('A3R534', 338), ('F2P1E0', 338), ('O00193', 97), ('O00303', 115), ('O43143', 4), ('O75347', 7), ('O75821', 168), ('O76021', 310), ('O76021', 407), ('O95218', 135), ('O95433', 246), ('P04075', 304), ('P04406', 323), ('P04908', 82), ('P04908', 100), ('P07737', 56), ('P08107', 49), ('P09496', 235), ('P09622', 482), ('P09669', 66), ('P10412', 25), ('P10515', 354), ('P12270', 2132), ('P18206', 823), ('P19623', 95), ('P20700', 90), ('P25398', 101), ('P27348', 138), ('P30101', 73), ('P31689', 124), ('P34932', 374), ('P35579', 1751), ('P35613', 170), ('P36507', 4), ('P42285', 128), ('P48637', 164), ('P50991', 527), ('P52272', 710), ('P52758', 107), ('P52907', 15), ('P61247', 51), ('P62736', 41), ('P62847', 20), ('P62906', 122), ('P63167', 4), ('P63220', 71), ('P78365', 123), ('P98175', 146), ('Q01518', 123), ('Q03405', 105), ('Q03405', 111), ('Q06830', 158), ('Q14568', 182), ('Q14839', 390), ('Q14974', 852), ('Q15029', 495), ('Q15050', 220), ('Q16576', 257), ('Q16778', 100),

In [10]:
positive_proteases_txt = {"HAT":"S01-047_human_airway_trypsin-like_peptidase.tsv", "DESC1":"S01-021_DESC1_peptidase.tsv",  
                      "TMPRSS2":"S01-247_epitheliasin.tsv", "TMPRSS4":"S01-034_transmembrane_peptidase-serine_4.tsv"}

# Uniprot APIを用いてアミノ酸配列を取得するスクリプトを用意します．
https://seiyakukenkyusya.com/programming/collecting-uniprot-information/
上記のリンク先を参考にしました．

In [11]:
uid_eg = 'P78365'

def aaseq_from_uid(uid, substrate_turn, index):
    df = pd.DataFrame(np.arange(3).reshape(1, 3), columns=['uniprotKB_accession', 'function', 'sequence'], index=['protease'+str(substrate_turn)+'_substrate'+str(index)])

    for column_name in df:
        df[column_name] = df[column_name].astype(str)

    df['uniprotKB_accession'][0] = uid

    #display(df)    

    url = "https://www.uniprot.org/uniprot/" + uid + ".xml"
    f = urlopen(url)
    xml = f.read()
    root = etree.fromstring(xml)
    
    #以下のコードは下の説明を参照
    function = root.find('./entry/comment[@type="function"]', root.nsmap)
    if function==None:
        print("function was not detected.")
        pass
    else:
        df["function"][0] = function[0].text
        #print(function[0].text+"¥n")

    sequence = root.find('./entry/sequence', root.nsmap) 
    if sequence==None: 
        print("sequence was not detected.")
        pass 
    else: 
        df["sequence"][0] = sequence.text 
        #print(sequence.text+"¥n")

    #display(df) 
    #df0=pd.concat([df0, df], axis=0)
    #display(df0)
    display(df) 
    return df["sequence"][0]

# 取得したアミノ酸からP1, P1'を特定する．今回はP4部位を戻り値にした．

In [12]:
def get_cleave_point(fullaa, cleave_pattern):
    #cleave_point=-1
    p4=-1
    cleave_points = []

    print(cleave_pattern)
    cleave_pattern02 = re.sub("-", "[A-Z]{1}", cleave_pattern)
    print(cleave_pattern02)
    
    for w in range(len(fullaa)-8):
        if re.search(cleave_pattern02, fullaa[w:w+8]):
            cleave_point = w
            cleave_points.append(w)
            print(cleave_points)
            print("cleave_point: " + str(cleave_point))
            continue
    
    print(f'cleave_point_list: {cleave_points}')
    if cleave_points:
        p4 = cleave_points[0]
    if p4 == -1:
        print("there is no cleave_pattern.")
    else:
        pass
    return p4, cleave_points

In [13]:
cp = 0
cpl = []
clpn="[A-Z]{1}[A-Z]{1}ARTLLA"
aaa = aaseq_from_uid("Q6ZS99", 0, 0)
for w in range(len(aaa)-8):
    if re.search(clpn, aaa[w:w+8]) != None:
        cp = w
        cpl.append(w)
        print(cpl)
        print("cleave_point: " + str(cp))
        continue

,uniprotKB_accession,function,sequence
protease0_substrate0,Q6ZS99,Nucleolin is the major nucleolar protein of gr...,MVKLAKAGKNQGDPKKMAPPPKEVEEDSEDEEMSEDEEDDEDDDDD...


[282]
cleave_point: 282


In [64]:
print(cp)
print(cpl)

282
[282]


In [15]:
def get_negative_data_seq(fullaa, cleave_point_list):
    cleave_range_list = []
    negative_eight_aa_list = []
    
    for s in range(len(cleave_point_list)):
        point = cleave_point_list[s]
        for t in range(-7, 8, 1):
            cleave_range_list.append(point+t)
        print(cleave_range_list)
    
    for s in range(len(cleave_point_list)):
        np.random.seed(0)
        a = np.arange(len(fullaa)-8)
        np.random.shuffle(a)
        print(a)
        
        t = 0
        while 1:
            if a[t] not in cleave_range_list:
                rdm = a[t]
                break
            else:
                t = t + 1
        negative_eight_aa = fullaa[rdm:rdm+8]
        negative_eight_aa_list.append(negative_eight_aa)

    return negative_eight_aa_list

In [16]:
# マッチパターンが複数あることを探す関数．
def search_double_pattern(fullaa, cleave_pattern):
    cleave_point=-1
    match_pattern_count = 0
    if "-" in cleave_pattern:
        cleave_pattern02 = re.sub("-", "[A-Z]{1}", cleave_pattern)
        for i in range(len(fullaa)-7):
            if re.search(cleave_pattern02, fullaa[i:i+8]) != None:
                cleave_point = i
                match_pattern_count = match_pattern_count + 1
                print("cleave_point: " + str(cleave_point))
                continue
    else:
        for i in range(len(fullaa)-7):
            #print(fullaa[i:i+8])
            if fullaa[i:i+8] == cleave_pattern:
                cleave_point = i
                match_pattern_count = match_pattern_count + 1
                print("cleave_point: " + str(cleave_point))
                continue
    return match_pattern_count

# ファイルを各々読み込んで，Uniprot IDからアミノ酸配列を取得します．

## 注意!! Uniprot IDは正規表現で[A-Z]{1}[0-9A-Z]{5}のように表す．
だけど，文字列"MERNUM"とかがヒットするのはどう対処するのか．

In [17]:
def get_cleave_pattern(p4_column):
    seq = ""
    three_aa = ""
    for j in range(8):
        three_aa = str(df.iloc[i, p4_column+j])
        print(f'three_aa: {three_aa}')
        
        #print(one_aa) 
        if three_aa == "Ala":
            one_aa = "A"
        elif three_aa == "Cys":
            one_aa = "C"
        elif three_aa == "Asp":
            one_aa = "D"
        elif three_aa == "Glu":
            one_aa = "E"
        elif three_aa == "Phe":
            one_aa = "F"
        elif three_aa == "Gly":
            one_aa = "G"
        elif three_aa == "His":
            one_aa = "H"
        elif three_aa == "Ile":
            one_aa = "I"
        elif three_aa == "Lys":
            one_aa = "K"
        elif three_aa == "Leu":
            one_aa = "L"
        elif three_aa == "Met":
            one_aa = "M"
        elif three_aa == "Asn":
            one_aa = "N"
        elif three_aa == "Pro":
            one_aa = "P"
        elif three_aa == "Gln":
            one_aa = "Q"
        elif three_aa == "Arg":
            one_aa = "R"
        elif three_aa == "Ser":
            one_aa = "S"
        elif three_aa == "Thr":
            one_aa = "T"
        elif three_aa == "Val":
            one_aa = "V"
        elif three_aa == "Trp":
            one_aa = "W"
        elif three_aa == "Tyr":
            one_aa = "Y"
        else:
            print("This is not a major amino acid or \"- \".")
            print(f'This is {three_aa}.')
            one_aa = "-"
            pass 
        print(f'one_aa: {one_aa}')
        seq = seq + one_aa
    print("cleave_pattern: "+seq)
    return seq

In [18]:
def aa_charge(e, cleave_pattern):
    for j in range(8):   
        one_char_aa = cleave_pattern[j]
        if one_char_aa == "A":
            x_train[e][0][j][0] = charge["A"]
        elif one_char_aa == "C":
            x_train[e][0][j][0] = charge["C"]
        elif one_char_aa == "D":
            x_train[e][0][j][0] = charge["D"]
        elif one_char_aa == "E":
            x_train[e][0][j][0] = charge["E"]
        elif one_char_aa == "F":
            x_train[e][0][j][0] = charge["F"]
        elif one_char_aa == "G":
            x_train[e][0][j][0] = charge["G"]
        elif one_char_aa == "H":
            x_train[e][0][j][0] = charge["H"]
        elif one_char_aa == "I":
            x_train[e][0][j][0] = charge["I"]
        elif one_char_aa == "K":
            x_train[e][0][j][0] = charge["K"]
        elif one_char_aa == "L":
            x_train[e][0][j][0] = charge["L"]
        elif one_char_aa == "M":
            x_train[e][0][j][0] = charge["M"]
        elif one_char_aa == "N":
            x_train[e][0][j][0] = charge["N"]
        elif one_char_aa == "P":
            x_train[e][0][j][0] = charge["P"]
        elif one_char_aa == "Q":
            x_train[e][0][j][0] = charge["Q"]
        elif one_char_aa == "R":
            x_train[e][0][j][0] = charge["R"]
        elif one_char_aa == "S":
            x_train[e][0][j][0] = charge["S"]
        elif one_char_aa == "T":
            x_train[e][0][j][0] = charge["T"]
        elif one_char_aa == "V":
            x_train[e][0][j][0] = charge["V"]
        elif one_char_aa == "W":
            x_train[e][0][j][0] = charge["W"]
        elif one_char_aa == "Y":
            x_train[e][0][j][0] = charge["Y"]
        else:
            pass

In [19]:
def aa_hypho(e, cleave_pattern):
    for j in range(8):
        one_char_aa = cleave_pattern[j]
        if one_char_aa == "A":
            x_train[e][1][j][0] = hydrophobicity["A"]
        elif one_char_aa == "C":
            x_train[e][1][j][0] = hydrophobicity["C"]
        elif one_char_aa == "D":
            x_train[e][1][j][0] = hydrophobicity["D"]
        elif one_char_aa == "E":
            x_train[e][1][j][0] = hydrophobicity["E"]
        elif one_char_aa == "F":
            x_train[e][1][j][0] = hydrophobicity["F"]
        elif one_char_aa == "G":
            x_train[e][1][j][0] = hydrophobicity["G"]
        elif one_char_aa == "H":
            x_train[e][1][j][0] = hydrophobicity["H"]
        elif one_char_aa == "I":
            x_train[e][1][j][0] = hydrophobicity["I"]
        elif one_char_aa == "K":
            x_train[e][1][j][0] = hydrophobicity["K"]
        elif one_char_aa == "L":
            x_train[e][1][j][0] = hydrophobicity["L"]
        elif one_char_aa == "M":
            x_train[e][1][j][0] = hydrophobicity["M"]
        elif one_char_aa == "N":
            x_train[e][1][j][0] = hydrophobicity["N"]
        elif one_char_aa == "P":
            x_train[e][1][j][0] = hydrophobicity["P"]
        elif one_char_aa == "Q":
            x_train[e][1][j][0] = hydrophobicity["Q"]
        elif one_char_aa == "R":
            x_train[e][1][j][0] = hydrophobicity["R"]
        elif one_char_aa == "S":
            x_train[e][1][j][0] = hydrophobicity["S"]
        elif one_char_aa == "T":
            x_train[e][1][j][0] = hydrophobicity["T"]
        elif one_char_aa == "V":
            x_train[e][1][j][0] = hydrophobicity["V"]
        elif one_char_aa == "W":
            x_train[e][1][j][0] = hydrophobicity["W"]
        elif one_char_aa == "Y":
            x_train[e][1][j][0] = hydrophobicity["Y"]
        else:
            pass    

In [20]:
#https://trade-and-develop.hatenablog.com/entry/2017/02/23/021119
cur = conn.cursor(buffered=True)

In [21]:
#adding secoundary structure
def add_secondary_structure(e, uid, p4, cleave_pattern):
    uid = [uid]
    cur.execute("SELECT substrate_2d FROM substrate_2d where uniprot_acc=(%s);", uid)  
    ss = cur.fetchall()
    if not ss:
        print("substrate_2d is empty.")
        return 0    
    else:
        print(ss)
    ssl=list(ss[0][0])
    
    for j in range(8):
        one_char_aa = cleave_pattern[j]
        if p4-1 +j < 0:
            continue
        if p4-1+j > len(ssl)-1:
            return 0
        if ssl[p4-1 + j] == 'a':
            x_train[e][2][j][0]= 1
        elif ssl[p4-1 + j] =='b':
            x_train[e][3][j][0] = 1
        else:
            pass  
    return 1

In [23]:
#https://www.sigmaaldrich.com/JP/ja/technical-documents/technical-article/protein-biology/protein-structural-analysis/amino-acid-reference-chart
df_aap = pd.read_table('./amino_acids_properties/amino_acid_info_merck.tsv', index_col=0)
print(df_aap)

                  3Letter 1Letter  MolecularWeight MolecularFormula  \
name                                                                  
Alanine               Ala       A            89.10          C3H7NO2   
Arginine              Arg       R           174.20        C6H14N4O2   
Asparagine            Asn       N           132.12         C4H8N2O3   
Aspartic acid         Asp       D           133.11          C4H7NO4   
Cysteine              Cys       C           121.16         C3H7NO2S   
Glutamic acid         Glu       E           147.13          C5H9NO4   
Glutamine             Gln       Q           146.15        C5H10N2O3   
Glycine               Gly       G            75.07          C2H5NO2   
Histidine             His       H           155.16         C6H9N3O2   
Hydroxyproline        Hyp       O           131.13          C5H9NO3   
Isoleucine            Ile       I           131.18         C6H13NO2   
Leucine               Leu       L           131.18         C6H13NO2   
Lysine

In [24]:
def aa_properties(e, cleave_pattern):
    for j in range(8):
        one_char_aa = cleave_pattern[j]
        if one_char_aa == "A":
            x_train[e][4][j][0] = df_aap.at['Alanine', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Alanine', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Alanine', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Alanine', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Alanine', 'pl']
            x_train[e][9][j][0] = df_aap.at['Alanine', 'C']
            x_train[e][10][j][0] = df_aap.at['Alanine', 'H']
            x_train[e][11][j][0] = df_aap.at['Alanine', 'N']
            x_train[e][12][j][0] = df_aap.at['Alanine', 'O']
            x_train[e][13][j][0] = df_aap.at['Alanine', 'S']
        elif one_char_aa == "C":
            #x_train[e][12][j][0][2] = hydrophobicity["C"]
            x_train[e][4][j][0] = df_aap.at['Cysteine', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Cysteine', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Cysteine', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Cysteine', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Cysteine', 'pl']
            x_train[e][9][j][0] = df_aap.at['Cysteine', 'C']
            x_train[e][10][j][0] = df_aap.at['Cysteine', 'H']
            x_train[e][11][j][0] = df_aap.at['Cysteine', 'N']
            x_train[e][12][j][0] = df_aap.at['Cysteine', 'O']
            x_train[e][13][j][0] = df_aap.at['Cysteine', 'S']
        elif one_char_aa == "D":
            #x_train[e][15][j][0][2] = hydrophobicity["D"]
            x_train[e][4][j][0] = df_aap.at['Aspartic acid', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Aspartic acid', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Aspartic acid', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Aspartic acid', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Aspartic acid', 'pl']
            x_train[e][9][j][0] = df_aap.at['Aspartic acid', 'C']
            x_train[e][10][j][0] = df_aap.at['Aspartic acid', 'H']
            x_train[e][11][j][0] = df_aap.at['Aspartic acid', 'N']
            x_train[e][12][j][0] = df_aap.at['Aspartic acid', 'O']
            x_train[e][13][j][0] = df_aap.at['Aspartic acid', 'S']
        elif one_char_aa == "E":
            #x_train[e][16][j][0][2] = hydrophobicity["E"]
            x_train[e][4][j][0] = df_aap.at['Glutamic acid', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Glutamic acid', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Glutamic acid', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Glutamic acid', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Glutamic acid', 'pl']
            x_train[e][9][j][0] = df_aap.at['Glutamic acid', 'C']
            x_train[e][10][j][0] = df_aap.at['Glutamic acid', 'H']
            x_train[e][11][j][0] = df_aap.at['Glutamic acid', 'N']
            x_train[e][12][j][0] = df_aap.at['Glutamic acid', 'O']
            x_train[e][13][j][0] = df_aap.at['Glutamic acid', 'S']
        elif one_char_aa == "F":
            #x_train[e][7][j][0][2] = hydrophobicity["F"]
            x_train[e][4][j][0] = df_aap.at['Phenylalanine', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Phenylalanine', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Phenylalanine', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Phenylalanine', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Phenylalanine', 'pl']
            x_train[e][9][j][0] = df_aap.at['Phenylalanine', 'C']
            x_train[e][10][j][0] = df_aap.at['Phenylalanine', 'H']
            x_train[e][11][j][0] = df_aap.at['Phenylalanine', 'N']
            x_train[e][12][j][0] = df_aap.at['Phenylalanine', 'O']
            x_train[e][13][j][0] = df_aap.at['Phenylalanine', 'S']
        elif one_char_aa == "G":
            #x_train[e][0][j][0][2] = hydrophobicity["G"]
            x_train[e][4][j][0] = df_aap.at['Glycine', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Glycine', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Glycine', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Glycine', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Glycine', 'pl']
            x_train[e][9][j][0] = df_aap.at['Glycine', 'C']
            x_train[e][10][j][0] = df_aap.at['Glycine', 'H']
            x_train[e][11][j][0] = df_aap.at['Glycine', 'N']
            x_train[e][12][j][0] = df_aap.at['Glycine', 'O']
            x_train[e][13][j][0] = df_aap.at['Glycine', 'S']
        elif one_char_aa == "H":
            #x_train[e][19][j][0][2] = hydrophobicity["H"]
            x_train[e][4][j][0] = df_aap.at['Histidine', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Histidine', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Histidine', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Histidine', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Histidine', 'pl']
            x_train[e][9][j][0] = df_aap.at['Histidine', 'C']
            x_train[e][10][j][0] = df_aap.at['Histidine', 'H']
            x_train[e][11][j][0] = df_aap.at['Histidine', 'N']
            x_train[e][12][j][0] = df_aap.at['Histidine', 'O']
            x_train[e][13][j][0] = df_aap.at['Histidine', 'S']
        elif one_char_aa == "I":
            #x_train[e][5][j][0][2] = hydrophobicity["I"]
            x_train[e][4][j][0] = df_aap.at['Isoleucine', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Isoleucine', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Isoleucine', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Isoleucine', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Isoleucine', 'pl']
            x_train[e][9][j][0] = df_aap.at['Isoleucine', 'C']
            x_train[e][10][j][0] = df_aap.at['Isoleucine', 'H']
            x_train[e][11][j][0] = df_aap.at['Isoleucine', 'N']
            x_train[e][12][j][0] = df_aap.at['Isoleucine', 'O']
            x_train[e][13][j][0] = df_aap.at['Isoleucine', 'S']
        elif one_char_aa == "K":
            #x_train[e][17][j][0][2] = hydrophobicity["K"]
            x_train[e][4][j][0] = df_aap.at['Lysine', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Lysine', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Lysine', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Lysine', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Lysine', 'pl']
            x_train[e][9][j][0] = df_aap.at['Lysine', 'C']
            x_train[e][10][j][0] = df_aap.at['Lysine', 'H']
            x_train[e][11][j][0] = df_aap.at['Lysine', 'N']
            x_train[e][12][j][0] = df_aap.at['Lysine', 'O']
            x_train[e][13][j][0] = df_aap.at['Lysine', 'S']
        elif one_char_aa == "L":
            #x_train[e][4][j][0][2] = hydrophobicity["L"]
            x_train[e][4][j][0] = df_aap.at['Leucine', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Leucine', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Leucine', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Leucine', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Leucine', 'pl']
            x_train[e][9][j][0] = df_aap.at['Leucine', 'C']
            x_train[e][10][j][0] = df_aap.at['Leucine', 'H']
            x_train[e][11][j][0] = df_aap.at['Leucine', 'N']
            x_train[e][12][j][0] = df_aap.at['Leucine', 'O']
            x_train[e][13][j][0] = df_aap.at['Leucine', 'S']
        elif one_char_aa == "M":
            #x_train[e][6][j][0][2] = hydrophobicity["M"]
            x_train[e][4][j][0] = df_aap.at['Methionine', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Methionine', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Methionine', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Methionine', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Methionine', 'pl']
            x_train[e][9][j][0] = df_aap.at['Methionine', 'C']
            x_train[e][10][j][0] = df_aap.at['Methionine', 'H']
            x_train[e][11][j][0] = df_aap.at['Methionine', 'N']
            x_train[e][12][j][0] = df_aap.at['Methionine', 'O']
            x_train[e][13][j][0] = df_aap.at['Methionine', 'S']
        elif one_char_aa == "N":
            #x_train[e][13][j][0][2] = hydrophobicity["N"]
            x_train[e][4][j][0] = df_aap.at['Asparagine', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Asparagine', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Asparagine', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Asparagine', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Asparagine', 'pl']
            x_train[e][9][j][0] = df_aap.at['Asparagine', 'C']
            x_train[e][10][j][0] = df_aap.at['Asparagine', 'H']
            x_train[e][11][j][0] = df_aap.at['Asparagine', 'N']
            x_train[e][12][j][0] = df_aap.at['Asparagine', 'O']
            x_train[e][13][j][0] = df_aap.at['Asparagine', 'S']
        elif one_char_aa == "P":
            #x_train[e][1][j][0][2] = hydrophobicity["P"]
            x_train[e][4][j][0] = df_aap.at['Proline', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Proline', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Proline', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Proline', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Proline', 'pl']
            x_train[e][9][j][0] = df_aap.at['Proline', 'C']
            x_train[e][10][j][0] = df_aap.at['Proline', 'H']
            x_train[e][11][j][0] = df_aap.at['Proline', 'N']
            x_train[e][12][j][0] = df_aap.at['Proline', 'O']
            x_train[e][13][j][0] = df_aap.at['Proline', 'S']
        elif one_char_aa == "Q":
            #x_train[e][14][j][0][2] = hydrophobicity["Q"]
            x_train[e][4][j][0] = df_aap.at['Glutamine', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Glutamine', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Glutamine', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Glutamine', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Glutamine', 'pl']
            x_train[e][9][j][0] = df_aap.at['Glutamine', 'C']
            x_train[e][10][j][0] = df_aap.at['Glutamine', 'H']
            x_train[e][11][j][0] = df_aap.at['Glutamine', 'N']
            x_train[e][12][j][0] = df_aap.at['Glutamine', 'O']
            x_train[e][13][j][0] = df_aap.at['Glutamine', 'S']
        elif one_char_aa == "R":
            #x_train[e][18][j][0][2] = hydrophobicity["R"]
            x_train[e][4][j][0] = df_aap.at['Arginine', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Arginine', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Arginine', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Arginine', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Arginine', 'pl']
            x_train[e][9][j][0] = df_aap.at['Arginine', 'C']
            x_train[e][10][j][0] = df_aap.at['Arginine', 'H']
            x_train[e][11][j][0] = df_aap.at['Arginine', 'N']
            x_train[e][12][j][0] = df_aap.at['Arginine', 'O']
            x_train[e][13][j][0] = df_aap.at['Arginine', 'S']
        elif one_char_aa == "S":
            #x_train[e][10][j][0][2] = hydrophobicity["S"]
            x_train[e][4][j][0] = df_aap.at['Serine', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Serine', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Serine', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Serine', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Serine', 'pl']
            x_train[e][9][j][0] = df_aap.at['Serine', 'C']
            x_train[e][10][j][0] = df_aap.at['Serine', 'H']
            x_train[e][11][j][0] = df_aap.at['Serine', 'N']
            x_train[e][12][j][0] = df_aap.at['Serine', 'O']
            x_train[e][13][j][0] = df_aap.at['Serine', 'S']
        elif one_char_aa == "T":
            #x_train[e][11][j][0][2] = hydrophobicity["T"]
            x_train[e][4][j][0] = df_aap.at['Threonine', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Threonine', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Threonine', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Threonine', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Threonine', 'pl']
            x_train[e][9][j][0] = df_aap.at['Threonine', 'C']
            x_train[e][10][j][0] = df_aap.at['Threonine', 'H']
            x_train[e][11][j][0] = df_aap.at['Threonine', 'N']
            x_train[e][12][j][0] = df_aap.at['Threonine', 'O']
            x_train[e][13][j][0] = df_aap.at['Threonine', 'S']
        elif one_char_aa == "V":
            #x_train[e][3][j][0][2] = hydrophobicity["V"]
            x_train[e][4][j][0] = df_aap.at['Valine', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Valine', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Valine', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Valine', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Valine', 'pl']
            x_train[e][9][j][0] = df_aap.at['Valine', 'C']
            x_train[e][10][j][0] = df_aap.at['Valine', 'H']
            x_train[e][11][j][0] = df_aap.at['Valine', 'N']
            x_train[e][12][j][0] = df_aap.at['Valine', 'O']
            x_train[e][13][j][0] = df_aap.at['Valine', 'S']
        elif one_char_aa == "W":
            #x_train[e][9][j][0][2] = hydrophobicity["W"]
            x_train[e][4][j][0] = df_aap.at['Tryptophan', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Tryptophan', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Tryptophan', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Tryptophan', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Tryptophan', 'pl']
            x_train[e][9][j][0] = df_aap.at['Tryptophan', 'C']
            x_train[e][10][j][0] = df_aap.at['Tryptophan', 'H']
            x_train[e][11][j][0] = df_aap.at['Tryptophan', 'N']
            x_train[e][12][j][0] = df_aap.at['Tryptophan', 'O']
            x_train[e][13][j][0] = df_aap.at['Tryptophan', 'S']
        elif one_char_aa == "Y":
            #x_train[e][8][j][0][2] = hydrophobicity["Y"]
            x_train[e][4][j][0] = df_aap.at['Tyrosine', 'MolecularWeight']
            x_train[e][5][j][0] = df_aap.at['Tyrosine', 'ResidueWeight_without_water']
            x_train[e][6][j][0] = df_aap.at['Tyrosine', 'pKa']
            x_train[e][7][j][0] = df_aap.at['Tyrosine', 'pKb']
            x_train[e][8][j][0] = df_aap.at['Tyrosine', 'pl']
            x_train[e][9][j][0] = df_aap.at['Tyrosine', 'C']
            x_train[e][10][j][0] = df_aap.at['Tyrosine', 'H']
            x_train[e][11][j][0] = df_aap.at['Tyrosine', 'N']
            x_train[e][12][j][0] = df_aap.at['Tyrosine', 'O']
            x_train[e][13][j][0] = df_aap.at['Tyrosine', 'S']
        else:
            pass    

In [25]:
def get_uid(m):
    for j in range(4):
        if re.match(r'^[A-Z]{1}[0-9A-Z]{5}$', str(df.iloc[i+m, j])):
            result = re.match(r'^[A-Z]{1}[0-9A-Z]{5}$', str(df.iloc[i+m, j]))
            uniprot_id = result.group(0)
            print(f'uniprot_id: {uniprot_id}')
            break
        else:
            uniprot_id = "null"
            print(f'uniprot_id: null')
    return uniprot_id

# main処理（creating positive data ）

In [26]:
# 各アミノ酸について電荷と疎水性の値を定義
charge = {"A":0., "C":-0.0735876, "D":-0.9994991, "E":-0.9987427, "F":0., "G":0., "H":0.0593509, "I":0.,
          "K":0.9997489, "L":0., "M":0., "N":0., "P":0., "Q":0., "R":0.9999950, "S":0., "T":0., "V":0., "W":0., "Y":-0.0001995}
hydrophobicity = {"A":0.1630295, "C":-0.2554557, "D":-0.9794608, "E":-0.7458280, "F":0.9152760,
                  "G":-0.2554557, "H":-0.7869063, "I":0.8510911, "K":-0.8510911, "L":1.,
                  "M":0.7227214, "N":-1., "P":-0.0860077, "Q":-0.8305520, "R":-0.7021823,
                  "S":-0.4685494, "T":-0.3838254, "V":0.6816431, "W":0.8305520, "Y":0.5738126}

In [71]:
count = 0
protease_turn = 0
consecutive_uid = 0
uid_list = []
flag = 0
substrate_uniprot_id_list = []
check = 0
total_subs_count = 0
subs_sum = 0
positive_data_dict = {}

for key_name in positive_proteases_txt:
    #filename = "./trainingdata(positive)/" + filename
    print("==================================protease"+str(count)+"=======================================")
    filename = positive_proteases_txt[key_name]
    path = f'../trainingdata(positive)/{filename}'
    df = pd.read_csv(path, sep='\t')
    display(df)
    
    x_train = np.zeros([len(df), 14, 8, 1])
    print("len(x_train):"+str(len(x_train))+"\n")

    df_cleave_pattern=pd.DataFrame(
        data={'uniprot_id':[], 'cleave_pattern':[]}
    )
    display(df_cleave_pattern)
    for i in range(len(df)):
        print("-------------------protease "+str(protease_turn)+"substrate"+str(i)+"---------------------")
        total_subs_count = total_subs_count + 1
        print("total_subs_count: "+str(total_subs_count))
        secondory_structure = "null"
        

        
        #get Uniprot ID
        uniprot_id = get_uid(0)
        
        #get the position of p4 column
        for j in range(5):
            #print(f'j: {j}')
            if re.match(r'^(N)*(S)*(P)*$', str(df.iloc[i, j])):
                p4_column = j + 2
                print(f'p4_column: {p4_column}')
                break
        
        #func1
        cleave_pattern = ""
        cleave_pattern = get_cleave_pattern(p4_column)       

        uid = uniprot_id 
        uid_list = []
        uid = str(uid)
        uid_list.append(uid)
        substrate_uniprot_id_list.append(uid)


        print("uid:" + uid)
        if uid == "null":
            print("This substrate does not have Uniprot ID.")
            pass
        else:
            fullaa = aaseq_from_uid(uid, protease_turn, i)
            print(f'fullaa_len: {len(fullaa)}')
            
            

        p4, cleave_point_list = get_cleave_point(fullaa, cleave_pattern)
        if p4 == -1:
            print(f'Cleave point was not detected.')
            continue
        print(f'p4: {str(p4)}')

        mpc = search_double_pattern(fullaa, cleave_pattern)
        print("cleave_site_count: "+str(mpc))
        if mpc  > 1:
            flag = flag +1
        
        #func2
        #aa_onezero()
        
        #func3
        aa_charge(i, cleave_pattern)

        #func4
        aa_hypho(i, cleave_pattern)


        #func5
        if uid == "null":
            print("secondary_structure information is none, because uid is null.")
            pass
        else:
            secondary_str = add_secondary_structure(i, uid, p4, cleave_pattern)
            if secondary_str == 0:
                secondary_structure = "null"
            else:
                secondary_structure = "not_null"

        #func6
        aa_properties(i, cleave_pattern)

        positive_data_dict[check] = [total_subs_count, protease_turn, uid, cleave_pattern, secondory_structure, x_train[i]]
        check = check + 1
        print("check: "+str(check))
        print("total_subs_count: "+str(total_subs_count))     
        
        df_cleave_pattern.loc[f'{i}'] = [uid, cleave_pattern]
        display(df_cleave_pattern)
        
        positive_data_dict[check] = [str(total_subs_count), str(protease_turn), uid, cleave_pattern, secondory_structure, x_train[i]]
        
    filename = f'./trainingdata/cleave_pattern_one_letter_aa_{key_name}.csv'
    df_cleave_pattern.to_csv(filename)
    
    print("The number of substrates that had multiple same cleavage sites: "+str(flag))
    np.save('./saving_ndarray/x_train/positive/x_train_positive_protease'+str(protease_turn), x_train)
    protease_turn = protease_turn + 1
    
    if count == 0:
        x_train_positive_all=x_train
        
    else:
        x_train_positive_all = np.append(x_train_positive_all, x_train, 0)
    subs_sum = subs_sum + len(df)
    count = count + 1
    
#print("consecutive_uid: "+str(consecutive_uid))
print("subs_sum: "+str(subs_sum))
print("total_subs_count: "+str(total_subs_count))
print(f"check: {check}")

print("END")

==================================protease0=======================================


,Substrate,Uniprot,Residue range,Cleavage Site,Cleavage type,Evidence,P4,P3,P2,P1,P1',P2',P3',P4',Reference,CutDB,MERNUM
0,P78365,109-134,peptide-Ser123+Ser-peptide,N,MS,Asn,Ser,Ala,Ser,Ser,Ile,Ala,Gly,"Barr� et al., 2014",NaN,NaN,NaN
1,Q6ZS99,285-301,AR+TLLAKNLPYKVTQDE,N,MS,-,-,Ala,Arg,Thr,Leu,Leu,Ala,"Barr� et al., 2014",NaN,NaN,NaN
2,116 kDa U5 small nuclear ribonucleoprotein com...,Q15029,487-512,peptide-Arg495+Val-peptide,N,MS,Ala,Phe,Gly,Arg,Val,Leu,Ser,Gly,"Barr� et al., 2014",NaN,NaN
3,14-3-3 protein theta,P27348,138-151,R+KQTIDNSQGAYQE,N,MS,-,-,-,Arg,Lys,Gln,Thr,Ile,"Barr� et al., 2014",NaN,NaN
4,40S ribosomal protein S12,P25398,98-117,GKPR+KVVGCSCVVVKDYGKE,N,MS,Gly,Lys,Pro,Arg,Lys,Val,Val,Gly,"Barr� et al., 2014",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Urokinase plasminogen activator surface receptor,Q03405,23-305,peptide-Arg111+Ser-peptide,P,NT,Thr,Tyr,Ser,Arg,Ser,Arg,Tyr,Leu,"Beaufort et al., 2007",NaN,NaN
96,Vimentin,Q8N850,157-172,LR+RQVDQLTNDKARVE,N,MS,-,-,Leu,Arg,Arg,Gln,Val,Asp,"Barr� et al., 2014",NaN,NaN
97,Vinculin,P18206,820-833,SGYR+ILGAVAKVRE,N,MS,Ser,Gly,Tyr,Arg,Ile,Leu,Gly,Ala,"Barr� et al., 2014",NaN,NaN
98,von Willebrand factor A domain-containing prot...,Q5TIE3,990-1007,GPQRSLATNT+LSSMKASE,N,MS,Ala,Thr,Asn,Thr,Leu,Ser,Ser,Met,"Barr� et al., 2014",NaN,NaN


len(x_train):100



,uniprot_id,cleave_pattern


-------------------protease 0substrate0---------------------
total_subs_count: 1
uniprot_id: P78365
p4_column: 5
three_aa: Asn
one_aa: N
three_aa: Ser
one_aa: S
three_aa: Ala
one_aa: A
three_aa: Ser
one_aa: S
three_aa: Ser
one_aa: S
three_aa: Ile
one_aa: I
three_aa: Ala
one_aa: A
three_aa: Gly
one_aa: G
cleave_pattern: NSASSIAG
uid:P78365
function was not detected.


...

-------------------protease 3substrate9---------------------
total_subs_count: 229
uniprot_id: null
uniprot_id: P37091
p4_column: 6
three_aa: Leu
one_aa: L
three_aa: Asn
one_aa: N
three_aa: Gln
one_aa: Q
three_aa: Arg
one_aa: R
three_aa: Ser
one_aa: S
three_aa: Ile
one_aa: I
three_aa: Met
one_aa: M
three_aa: Glu
one_aa: E
cleave_pattern: LNQRSIME
uid:P37091


,uniprotKB_accession,function,sequence
protease3_substrate9,P37091,Sodium permeable non-voltage-sensitive ion cha...,MAPGEKIKAKIKKNLPVRGPQAPTIKDLMHWYCMNTNTHGCRRIVV...


fullaa_len: 650
LNQRSIME
LNQRSIME
[511]
cleave_point: 511
cleave_point_list: [511]
p4: 511
cleave_point: 511
cleave_site_count: 1
substrate_2d is empty.
check: 206
total_subs_count: 229


,uniprot_id,cleave_pattern
0,P37089,NSDKLVFP
1,P37089,NPYRYTEI
2,P37089,TEIKEELE
3,P37089,ELDRITEQ
4,P37089,RSSRDLLG
5,P37089,RTARSGSS
6,P37089,LNYKTNSE
7,P37090,FNYRTIEE
8,P37091,RKRREAGS
9,P37091,LNQRSIME


The number of substrates that had multiple same cleavage sites: 1
subs_sum: 229
total_subs_count: 229
check: 206
END


In [ ]:
positive_data_dict

In [1]:
import csv

field_name = ['total_subs_count', 'protease_turn', 'uniprot_id', 'cleave_pattern', 'secondory_structure', 'x_train[i]']
with open(r'./saving_ndarray/x_train/positive_data_dict_contents.csv', 'w', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(positive_data_dic)
    

FileNotFoundError: [Errno 2] No such file or directory: './saving_ndarray/x_train/positive_data_dict_contents.csv'

In [36]:
import pickle

#my_dict = { 'Apple': 4, 'Banana': 2, 'Orange': 6, 'Grapes': 11}

with open("./saving_ndarray/x_train/positive_data_dic.pkl", "wb") as tf:
    pickle.dump(positive_data_dic,tf)

In [1]:
import pickle

with open("./saving_ndarray/x_train/positive_data_dic.pkl", "wb") as tf:
    positive_data_dictionary = pickle.load(tf)

print(positive_data_dictionary.item())

FileNotFoundError: [Errno 2] No such file or directory: './saving_ndarray/x_train/positive_data_dic.pkl'

In [38]:
print(len(positive_data_dic))

207


ここの計算は10分あれば終わる．

In [39]:
#save positive
np.save('./saving_ndarray/x_train/x_train_all_only_positive', x_train_positive_all)

In [2]:
#load positive
x_train_positive_all = np.load('./saving_ndarray/x_train/x_train_all_only_positive.npy')

In [40]:
print(len(x_train_positive_all))

229


In [128]:
#print(x_train_positive_all_dic)

In [192]:
#スパース行列を数える．
del_list = []
for i in range(len(x_train_positive_all)):
    #print(x_train_all[i])
    a = x_train_positive_all[i]
    #print(a)
    
    zero_row_count = 0
    for j in range(14):
        print(a[j])
        if np.all(a[j]==0):
            zero_row_count = zero_row_count + 1
    if zero_row_count == 14:
        del_list.append(i)
    print("=====================================================")
    

#x_train_all = np.delete(x_train_all, del_list, 0)
print(x_train_positive_all)
print(len(x_train_positive_all))
print(len(del_list))
    
#https://engineeeer.com/check-numpy-array-all-zero/

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[-1.       ]
 [-0.4685494]
 [ 0.1630295]
 [-0.4685494]
 [-0.4685494]
 [ 0.8510911]
 [ 0.1630295]
 [-0.2554557]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[132.12]
 [105.09]
 [ 89.1 ]
 [105.09]
 [105.09]
 [131.18]
 [ 89.1 ]
 [ 75.07]]
[[114.11]
 [ 87.08]
 [ 71.08]
 [ 87.08]
 [ 87.08]
 [113.16]
 [ 71.08]
 [ 57.05]]
[[2.02]
 [2.21]
 [2.34]
 [2.21]
 [2.21]
 [2.36]
 [2.34]
 [2.34]]
[[8.8 ]
 [9.15]
 [9.69]
 [9.15]
 [9.15]
 [9.6 ]
 [9.69]
 [9.6 ]]
[[5.41]
 [5.68]
 [6.  ]
 [5.68]
 [5.68]
 [6.02]
 [6.  ]
 [5.97]]
[[4.]
 [3.]
 [3.]
 [3.]
 [3.]
 [6.]
 [3.]
 [2.]]
[[ 8.]
 [ 7.]
 [ 7.]
 [ 7.]
 [ 7.]
 [13.]
 [ 7.]
 [ 5.]]
[[2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[3.]
 [3.]
 [2.]
 [3.]
 [3.]
 [2.]
 [2.]
 [2.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.      ]
 [0.      ]
 [0.      ]
 [0.999995]
 [0.      ]
 [0.      ]
 [0.      ]
 [0.      ]]
[[ 0.       ]
 [ 0.       ]
 [ 0.

 [2.34]]
[[9.11]
 [9.6 ]
 [9.15]
 [9.04]
 [9.17]
 [8.8 ]
 [9.1 ]
 [9.69]]
[[ 5.66]
 [ 5.98]
 [ 5.68]
 [10.76]
 [ 7.59]
 [ 5.41]
 [ 5.6 ]
 [ 6.  ]]
[[9.]
 [6.]
 [3.]
 [6.]
 [6.]
 [4.]
 [4.]
 [3.]]
[[11.]
 [13.]
 [ 7.]
 [14.]
 [ 9.]
 [ 8.]
 [ 9.]
 [ 7.]]
[[1.]
 [1.]
 [1.]
 [4.]
 [3.]
 [2.]
 [1.]
 [1.]]
[[3.]
 [2.]
 [3.]
 [2.]
 [2.]
 [3.]
 [3.]
 [2.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.       ]
 [0.9997489]
 [0.       ]
 [0.999995 ]
 [0.       ]
 [0.       ]
 [0.9997489]
 [0.       ]]
[[ 1.       ]
 [-0.8510911]
 [-0.0860077]
 [-0.7021823]
 [ 0.6816431]
 [-0.2554557]
 [-0.8510911]
 [ 0.1630295]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[131.18]
 [146.19]
 [115.13]
 [174.2 ]
 [117.15]
 [ 75.07]
 [146.19]
 [ 89.1 ]]
[[113.16]
 [128.18]
 [ 97.12]
 [156.19]
 [ 99.13]
 [ 57.05]
 [128.18]
 [ 71.08]]
[[2.36]
 [2.18]
 [1.99]
 [2.17]
 [2.32]
 [2.34]
 [2.18]
 [2.34]]
[[ 9.6 ]
 [ 8.95]
 [10.6 ]
 [ 9.04]
 [ 9.62]
 [ 9.6 ]
 [ 

[[ 87.08]
 [147.18]
 [101.11]
 [156.19]
 [ 57.05]
 [ 87.08]
 [115.09]
 [ 87.08]]
[[2.21]
 [1.83]
 [2.09]
 [2.17]
 [2.34]
 [2.21]
 [1.88]
 [2.21]]
[[9.15]
 [9.13]
 [9.1 ]
 [9.04]
 [9.6 ]
 [9.15]
 [9.6 ]
 [9.15]]
[[ 5.68]
 [ 5.48]
 [ 5.6 ]
 [10.76]
 [ 5.97]
 [ 5.68]
 [ 2.77]
 [ 5.68]]
[[3.]
 [9.]
 [4.]
 [6.]
 [2.]
 [3.]
 [4.]
 [3.]]
[[ 7.]
 [11.]
 [ 9.]
 [14.]
 [ 5.]
 [ 7.]
 [ 7.]
 [ 7.]]
[[1.]
 [1.]
 [1.]
 [4.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[3.]
 [2.]
 [3.]
 [2.]
 [2.]
 [3.]
 [4.]
 [3.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.       ]
 [0.       ]
 [0.       ]
 [0.999995 ]
 [0.       ]
 [0.       ]
 [0.9997489]
 [0.       ]]
[[ 0.1630295]
 [ 0.1630295]
 [ 1.       ]
 [-0.7021823]
 [ 1.       ]
 [ 0.6816431]
 [-0.8510911]
 [-0.0860077]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[ 89.1 ]
 [ 89.1 ]
 [131.18]
 [174.2 ]
 [131.18]
 [117.15]
 [146.19]
 [115.13]]
[[ 71.08]
 [ 71.08]
 [113.16]
 [156.19]
 [113.16]
 [ 99.13]
 [128.

In [193]:
#スパース行列を削除する．
del_list = []
for i in range(len(x_train_positive_all)):
    #print(x_train_all[i])
    a = x_train_positive_all[i]
    #print(a)
    
    zero_row_count = 0
    for j in range(14):
        print(a[j])
        if np.all(a[j]==0):
            zero_row_count = zero_row_count + 1
    if zero_row_count == 14:
        del_list.append(i)
    print("=====================================================")
    

x_train_positive_all = np.delete(x_train_positive_all, del_list, 0)
print(x_train_positive_all)
print(len(x_train_positive_all))
print(len(del_list))

#https://engineeeer.com/check-numpy-array-all-zero/

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[-1.       ]
 [-0.4685494]
 [ 0.1630295]
 [-0.4685494]
 [-0.4685494]
 [ 0.8510911]
 [ 0.1630295]
 [-0.2554557]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[132.12]
 [105.09]
 [ 89.1 ]
 [105.09]
 [105.09]
 [131.18]
 [ 89.1 ]
 [ 75.07]]
[[114.11]
 [ 87.08]
 [ 71.08]
 [ 87.08]
 [ 87.08]
 [113.16]
 [ 71.08]
 [ 57.05]]
[[2.02]
 [2.21]
 [2.34]
 [2.21]
 [2.21]
 [2.36]
 [2.34]
 [2.34]]
[[8.8 ]
 [9.15]
 [9.69]
 [9.15]
 [9.15]
 [9.6 ]
 [9.69]
 [9.6 ]]
[[5.41]
 [5.68]
 [6.  ]
 [5.68]
 [5.68]
 [6.02]
 [6.  ]
 [5.97]]
[[4.]
 [3.]
 [3.]
 [3.]
 [3.]
 [6.]
 [3.]
 [2.]]
[[ 8.]
 [ 7.]
 [ 7.]
 [ 7.]
 [ 7.]
 [13.]
 [ 7.]
 [ 5.]]
[[2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[3.]
 [3.]
 [2.]
 [3.]
 [3.]
 [2.]
 [2.]
 [2.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.      ]
 [0.      ]
 [0.      ]
 [0.999995]
 [0.      ]
 [0.      ]
 [0.      ]
 [0.      ]]
[[ 0.       ]
 [ 0.       ]
 [ 0.

 [181.19]]
[[114.11]
 [ 87.08]
 [ 57.05]
 [156.19]
 [ 71.08]
 [ 99.13]
 [101.11]
 [163.18]]
[[2.02]
 [2.21]
 [2.34]
 [2.17]
 [2.34]
 [2.32]
 [2.09]
 [2.2 ]]
[[8.8 ]
 [9.15]
 [9.6 ]
 [9.04]
 [9.69]
 [9.62]
 [9.1 ]
 [9.11]]
[[ 5.41]
 [ 5.68]
 [ 5.97]
 [10.76]
 [ 6.  ]
 [ 5.96]
 [ 5.6 ]
 [ 5.66]]
[[4.]
 [3.]
 [2.]
 [6.]
 [3.]
 [5.]
 [4.]
 [9.]]
[[ 8.]
 [ 7.]
 [ 5.]
 [14.]
 [ 7.]
 [11.]
 [ 9.]
 [11.]]
[[2.]
 [1.]
 [1.]
 [4.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[3.]
 [3.]
 [2.]
 [2.]
 [2.]
 [2.]
 [3.]
 [3.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[ 0.00000e+00]
 [-1.99500e-04]
 [ 0.00000e+00]
 [ 9.99995e-01]
 [ 0.00000e+00]
 [ 9.99995e-01]
 [-1.99500e-04]
 [ 0.00000e+00]]
[[-0.3838254]
 [ 0.5738126]
 [-0.4685494]
 [-0.7021823]
 [-0.4685494]
 [-0.7021823]
 [ 0.5738126]
 [ 1.       ]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[119.12]
 [181.19]
 [105.09]
 [174.2 ]
 [105.09]
 [174.2 ]
 [181.19]
 [131.18]]
[[101.11]
 [163.18]
 [ 87.08]
 

 [3.]]
[[10.]
 [10.]
 [ 7.]
 [14.]
 [14.]
 [ 9.]
 [ 7.]
 [ 7.]]
[[2.]
 [2.]
 [1.]
 [4.]
 [2.]
 [1.]
 [1.]
 [1.]]
[[3.]
 [3.]
 [2.]
 [2.]
 [2.]
 [3.]
 [2.]
 [3.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.      ]
 [0.      ]
 [0.      ]
 [0.999995]
 [0.999995]
 [0.      ]
 [0.      ]
 [0.      ]]
[[ 0.       ]
 [ 0.8510911]
 [ 0.915276 ]
 [-0.7021823]
 [-0.7021823]
 [ 0.915276 ]
 [-0.0860077]
 [ 0.6816431]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[  0.  ]
 [131.18]
 [165.19]
 [174.2 ]
 [174.2 ]
 [165.19]
 [115.13]
 [117.15]]
[[  0.  ]
 [113.16]
 [147.18]
 [156.19]
 [156.19]
 [147.18]
 [ 97.12]
 [ 99.13]]
[[0.  ]
 [2.36]
 [1.83]
 [2.17]
 [2.17]
 [1.83]
 [1.99]
 [2.32]]
[[ 0.  ]
 [ 9.6 ]
 [ 9.13]
 [ 9.04]
 [ 9.04]
 [ 9.13]
 [10.6 ]
 [ 9.62]]
[[ 0.  ]
 [ 6.02]
 [ 5.48]
 [10.76]
 [10.76]
 [ 5.48]
 [ 6.3 ]
 [ 5.96]]
[[0.]
 [6.]
 [9.]
 [6.]
 [6.]
 [9.]
 [5.]
 [5.]]
[[ 0.]
 [13.]
 [11.]
 [14.]
 [14.]
 [11.]
 [ 9.]
 [11.]]
[[

In [194]:
print(len(x_train_positive_all))

206


In [195]:
print(x_train_positive_all[100])

[[[ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 9.999950e-01]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]]

 [[-1.000000e+00]
  [-8.600770e-02]
  [-1.000000e+00]
  [-7.021823e-01]
  [ 6.816431e-01]
  [ 1.630295e-01]
  [-8.305520e-01]
  [-3.838254e-01]]

 [[ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]]

 [[ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]]

 [[ 1.321200e+02]
  [ 1.151300e+02]
  [ 1.321200e+02]
  [ 1.742000e+02]
  [ 1.171500e+02]
  [ 8.910000e+01]
  [ 1.461500e+02]
  [ 1.191200e+02]]

 [[ 1.141100e+02]
  [ 9.712000e+01]
  [ 1.141100e+02]
  [ 1.561900e+02]
  [ 9.913000e+01]
  [ 7.108000e+01]
  [ 1.281300e+02]
  [ 1.011100e+02]]

 [[ 2.020000e+00]
  [ 1.990000e+00]
  [ 2.020000e+00]
  [ 2.170000e+00]
  [ 2.320000e+00]
  [ 2.340000e+00]
  [ 2.170000e+00

In [196]:
positive_data_dic[100][3]

'NPNRVAQT'

In [41]:
five_cross = len(positive_data_dic)//5

In [42]:
print(five_cross)

41


https://note.nkmk.me/python-random-randrange-randint/

In [43]:
rand_list_5cross = random.sample(range(0, len(positive_data_dic), 1), k=five_cross)

In [44]:
print(rand_list_5cross)

[17, 69, 148, 99, 123, 32, 177, 64, 117, 86, 100, 25, 165, 144, 66, 195, 197, 188, 106, 181, 171, 22, 81, 73, 60, 55, 120, 122, 10, 71, 113, 175, 26, 140, 80, 9, 110, 21, 152, 109, 31]


In [45]:
df1 = pd.DataFrame(
    data={'check': rand_list_5cross}
)
 
df1

,check
0,17
1,69
2,148
3,99
4,123
5,32
6,177
7,64
8,117
9,86


In [46]:
df1.to_csv("./saving_ndarray/x_train/x_train_positive_all_5crossval.csv")

In [47]:
print(len(rand_list_5cross))

41


# 検証データpositive_5crossvalを作る．

In [48]:
count = 0
for i in rand_list_5cross:
    if count == 0:
        x_train_positive_all_5crossval = [positive_data_dic[i][5]]
    else:
        x_train_positive_all_5crossval = np.append(x_train_positive_all_5crossval, [positive_data_dic[i][5]], 0)
    count = count + 1
    print(positive_data_dic[i])
    

[21, 0, 'Q14839', 'TCPRAYHM', 'null', array([[[ 0.000000e+00],
        [-7.358760e-02],
        [ 0.000000e+00],
        [ 9.999950e-01],
        [ 0.000000e+00],
        [-1.995000e-04],
        [ 5.935090e-02],
        [ 0.000000e+00]],

       [[-3.838254e-01],
        [-2.554557e-01],
        [-8.600770e-02],
        [-7.021823e-01],
        [ 1.630295e-01],
        [ 5.738126e-01],
        [-7.869063e-01],
        [ 7.227214e-01]],

       [[ 0.000000e+00],
        [ 0.000000e+00],
        [ 0.000000e+00],
        [ 0.000000e+00],
        [ 0.000000e+00],
        [ 0.000000e+00],
        [ 0.000000e+00],
        [ 0.000000e+00]],

       [[ 0.000000e+00],
        [ 0.000000e+00],
        [ 0.000000e+00],
        [ 0.000000e+00],
        [ 0.000000e+00],
        [ 0.000000e+00],
        [ 0.000000e+00],
        [ 0.000000e+00]],

       [[ 1.191200e+02],
        [ 1.211600e+02],
        [ 1.151300e+02],
        [ 1.742000e+02],
        [ 8.910000e+01],
        [ 1.811900e+02],
    

In [49]:
print(count)

41


In [50]:
print(len(x_train_positive_all_5crossval))

41


In [51]:
print(x_train_positive_all_5crossval[0])

[[[ 0.000000e+00]
  [-7.358760e-02]
  [ 0.000000e+00]
  [ 9.999950e-01]
  [ 0.000000e+00]
  [-1.995000e-04]
  [ 5.935090e-02]
  [ 0.000000e+00]]

 [[-3.838254e-01]
  [-2.554557e-01]
  [-8.600770e-02]
  [-7.021823e-01]
  [ 1.630295e-01]
  [ 5.738126e-01]
  [-7.869063e-01]
  [ 7.227214e-01]]

 [[ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]]

 [[ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]]

 [[ 1.191200e+02]
  [ 1.211600e+02]
  [ 1.151300e+02]
  [ 1.742000e+02]
  [ 8.910000e+01]
  [ 1.811900e+02]
  [ 1.551600e+02]
  [ 1.492100e+02]]

 [[ 1.011100e+02]
  [ 1.031500e+02]
  [ 9.712000e+01]
  [ 1.561900e+02]
  [ 7.108000e+01]
  [ 1.631800e+02]
  [ 1.371400e+02]
  [ 1.312000e+02]]

 [[ 2.090000e+00]
  [ 1.960000e+00]
  [ 1.990000e+00]
  [ 2.170000e+00]
  [ 2.340000e+00]
  [ 2.200000e+00]
  [ 1.820000e+00

In [52]:
#print(x_train_positive_all[0])

In [53]:
#[i for i in dlist1 if i not in dlist2]
#
positive_all_list = set(list(range(0, len(positive_data_dic)))) - set(rand_list_5cross)
print(positive_all_list)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 18, 19, 20, 23, 24, 27, 28, 29, 30, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 61, 62, 63, 65, 67, 68, 70, 72, 74, 75, 76, 77, 78, 79, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 101, 102, 103, 104, 105, 107, 108, 111, 112, 114, 115, 116, 118, 119, 121, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 141, 142, 143, 145, 146, 147, 149, 150, 151, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 166, 167, 168, 169, 170, 172, 173, 174, 176, 178, 179, 180, 182, 183, 184, 185, 186, 187, 189, 190, 191, 192, 193, 194, 196, 198, 199, 200, 201, 202, 203, 204, 205, 206}


In [54]:
print(len(positive_all_list))

166


In [120]:
dlist1= list(range(0, len(positive_data_dic)))
dlist2 = rand_list_5cross
dlist3 = [i for i in dlist1 if i not in dlist2]
print(dlist3)
print(len(dlist3))

[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 13, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 32, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 50, 52, 53, 54, 56, 57, 60, 62, 63, 68, 69, 70, 71, 72, 73, 74, 75, 76, 79, 80, 81, 82, 83, 84, 86, 87, 88, 89, 90, 92, 93, 94, 96, 97, 99, 100, 103, 104, 105, 106, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 122, 124, 125, 126, 127, 128, 129, 130, 131, 135, 136, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 172, 173, 174, 176, 177, 178, 179, 180, 181, 182, 183, 184, 186, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 205]
165


# 訓練データpositive_allを作る．

In [55]:
count = 0
for i in positive_all_list:
    if count == 0:
        x_train_positive_all = [positive_data_dic[i][5]]
    else:
        x_train_positive_all = np.append(x_train_positive_all, [positive_data_dic[i][5]], 0)
    count = count + 1
    print(positive_data_dic[i])
    

[1, 0, 'P78365', 'NSASSIAG', 'null', array([[[  0.       ],
        [  0.       ],
        [  0.       ],
        [  0.       ],
        [  0.       ],
        [  0.       ],
        [  0.       ],
        [  0.       ]],

       [[ -1.       ],
        [ -0.4685494],
        [  0.1630295],
        [ -0.4685494],
        [ -0.4685494],
        [  0.8510911],
        [  0.1630295],
        [ -0.2554557]],

       [[  0.       ],
        [  0.       ],
        [  0.       ],
        [  0.       ],
        [  0.       ],
        [  0.       ],
        [  0.       ],
        [  0.       ]],

       [[  0.       ],
        [  0.       ],
        [  0.       ],
        [  0.       ],
        [  0.       ],
        [  0.       ],
        [  0.       ],
        [  0.       ]],

       [[132.12     ],
        [105.09     ],
        [ 89.1      ],
        [105.09     ],
        [105.09     ],
        [131.18     ],
        [ 89.1      ],
        [ 75.07     ]],

       [[114.11     ],
        [ 

In [56]:
x_train_positive_all

array([[[[ 0.       ],
         [ 0.       ],
         [ 0.       ],
         ...,
         [ 0.       ],
         [ 0.       ],
         [ 0.       ]],

        [[-1.       ],
         [-0.4685494],
         [ 0.1630295],
         ...,
         [ 0.8510911],
         [ 0.1630295],
         [-0.2554557]],

        [[ 0.       ],
         [ 0.       ],
         [ 0.       ],
         ...,
         [ 0.       ],
         [ 0.       ],
         [ 0.       ]],

        ...,

        [[ 2.       ],
         [ 1.       ],
         [ 1.       ],
         ...,
         [ 1.       ],
         [ 1.       ],
         [ 1.       ]],

        [[ 3.       ],
         [ 3.       ],
         [ 2.       ],
         ...,
         [ 2.       ],
         [ 2.       ],
         [ 2.       ]],

        [[ 0.       ],
         [ 0.       ],
         [ 0.       ],
         ...,
         [ 0.       ],
         [ 0.       ],
         [ 0.       ]]],


       [[[ 0.       ],
         [ 0.       ],
         [ 0. 

In [57]:
print(len(x_train_positive_all))

166


In [153]:
#len(positive_data_dic)

207

In [98]:
#for i in range(len(positive_data_dic)):
#    print(positive_data_dic[i])

In [93]:
#print(positive_data_dic[1])

In [92]:
#print(x_train_positive_all[119])

In [ ]:
#random.shuffle(x_train_positive_all)

check what sequence to try create validataion data.

In [ ]:
# check what sequence to try create validataion data.
print(x_train_positive_all)

In [ ]:
#x_train_all_5crossval = x_train_positive_all[:five_cross]

In [ ]:
print(x_train_all_5crossval)

In [ ]:
print(len(x_train_all_5crossval))

In [ ]:
#x_train_positive_all = np.delete(x_train_positive_all, slice(0, five_cross), axis=0)

In [ ]:
print(len(x_train_positive_all))

In [58]:
y_train_positive_all =  np.array([[1.0, 0.0]]*len(x_train_positive_all))

In [ ]:
print(len(y_train_all))

In [59]:
y_train_positive_all_5crossval  = np.array([[1.0, 0.0]]*len(x_train_positive_all_5crossval))

In [ ]:
print(len(y_train_all_5crossval))

In [65]:
print(substrate_uniprot_id_list)
print(len(substrate_uniprot_id_list))

['P78365', 'Q6ZS99', 'Q15029', 'P27348', 'P25398', 'P63220', 'P62847', 'P61247', 'P62906', 'null', 'null', 'P62736', 'O95433', 'Q01518', 'Q8IU61', 'Q96IR6', 'Q9BQ39', 'P35613', 'Q9NYF8', 'null', 'Q14839', 'P09496', 'P09669', 'P09622', 'P10515', 'P31689', 'Q9Y3Q2', 'P36507', 'P63167', 'O00303', 'O75821', 'P52907', 'Q92945', 'Q9UEV9', 'P04075', 'P48637', 'P04406', 'P08107', 'P34932', 'A3R530', 'A3R534', 'F2P1E0', 'P52272', 'Q86U45', 'Q9NNZ5', 'P10412', 'P04908', 'P04908', 'P04908', 'Q71UI9', 'Q16778', 'Q16576', 'Q14974', 'Q9NSK0', 'Q9UDE8', 'P20700', 'Q5CZI7', 'P35579', 'Q9UMX5', 'Q9Y266', 'Q9Y2X3', 'Q9BVL2', 'P12270', 'Q06830', 'Q9NSD9', 'Q9NSD9', 'Q9Y367', 'Q9Y367', 'P07737', 'P30101', 'Q96II9', 'Q14568', 'Q14568', 'O43143', 'Q96E76', 'Q96E76', 'P52758', 'Q8IY67', 'O76021', 'O76021', 'Q15050', 'P98175', 'Q8IYM1', 'O00193', 'P19623', 'Q9NTJ3', 'P42285', 'Q9UNK0', 'P50991', 'Q9ULW0', 'Q86V81', 'Q9BV44', 'Q8WZ71', 'O75347', 'Q03405', 'Q03405', 'Q8N850', 'P18206', 'Q5TIE3', 'O95218', 'Q9BQ

In [66]:
ulist_mece = set(substrate_uniprot_id_list)
print(ulist_mece)
print(len(ulist_mece))

{'Q9NZI8', 'P07195', 'P35579', 'O75915', 'Q99832', 'P08107', 'O76021', 'Q9UEQ5', 'Q96HK4', 'P09669', 'P20700', 'Q14974', 'Q9UNK0', 'P11021', 'P52272', 'P36507', 'Q07955', 'P68366', 'P10515', 'O75821', 'P04406', 'P34932', 'P04075', 'P15531', 'P09622', 'null', 'Q9BVL2', 'Q9ULW0', 'P52597', 'Q9UEG1', 'Q9NYF8', 'Q5TCJ2', 'Q9H009', 'P25398', 'Q9Y367', 'Q8N850', 'Q96IR6', 'Q8WXF1', 'P50991', 'Q9H7W5', 'P29401', 'P30101', 'Q86V81', 'O15393', 'Q96II9', 'Q5JVS0', 'O14776', 'Q9UMX5', 'Q8IU61', 'Q03405', 'P10412', 'Q9BQ39', 'Q9BQ02', 'B1AK88', 'Q99729', 'O14950', 'Q6GMP2', 'Q9Y5L4', 'P84157', 'P05783', 'Q6ZS99', 'Q5TIE3', 'O43143', 'Q9UDE8', 'O14979', 'P25787', 'Q06830', 'P09496', 'P62906', 'P10809', 'Q13442', 'Q9UEV9', 'O14561', 'P08758', 'Q9NSK0', 'P37089', 'Q16576', 'Q9Y266', 'Q99623', 'Q9NNZ5', 'Q15050', 'Q71UI9', 'P52294', 'Q8IY67', 'Q99778', 'P54577', 'Q7M4R4', 'P39748', 'P98175', 'P35613', 'O95433', 'P26599', 'Q92945', 'P16403', 'Q14568', 'Q9H6F5', 'Q8WZ71', 'P14854', 'P63167', 'Q5CZI7', '

In [62]:
print(len(x_train_positive_all))
print(len(x_train_positive_all_5crossval))

print(len(y_train_positive_all))
print(len(y_train_positive_all_5crossval))
print(subs_sum)

166
41
166
41
229


In [ ]:
for i in range(len(x_train_positive_all)):
    print(x_train_positive_all[i])

In [ ]:
for i in range(len(x_train_all_5crossval)):
    print(x_train_all_5crossval[i])

In [ ]:
for i in range(len(y_train_all)):
    print(y_train_all[0])

In [ ]:
for i in range(len(y_train_all_5crossval)):
    print(y_train_all_5crossval[i])

## 途中経過を保存する．

In [61]:
# saving numarray(ndarray)
np.save('./saving_ndarray/x_train/x_train_positive_all', x_train_positive_all)
np.save('./saving_ndarray/x_train/y_train_positive_all', y_train_positive_all)
np.save('./saving_ndarray/x_train/x_train_positive_all_ 5crossval', x_train_positive_all_5crossval)
np.save('./saving_ndarray/x_train/y_train_positive_all_5crossval', y_train_positive_all_5crossval)

In [63]:
print(len(x_train_positive_all))
print(len(x_train_all_5crossval))
print(len(y_train_all))
print(len(y_train_all_5crossval))
print(subs_sum)

166


NameError: name 'x_train_all_5crossval' is not defined

## データをロードする．

In [ ]:
#loding ndarray
print(type(np.load('../saving_ndarray/x_train/x_train_all_only_positive.npy')))
print(np.load('./saving_ndarray/x_train/x_train_all_only_positive.npy'))

x_train_positive_all = np.load('./saving_ndarray/x_train/x_train_all_only_positive.npy')
y_train_all = np.load('../saving_ndarray/x_train/y_train_all_only_positive.npy')
x_train_all_5crossval = np.load('./saving_ndarray/x_train/x_train_all_ 5crossval_only_positive.npy')
y_train_all_5crossval = np.load('./saving_ndarray/x_train/y_train_all_5crossval_only_positive.npy')

In [ ]:
print(len(x_train_positive_all))
print(len(x_train_all_5crossval))
print(len(y_train_all))
print(len(y_train_all_5crossval))
print(subs_sum)